<h1>機械学習APIの使用</h1>

まず、APIコンソールにアクセスし、左側のメニューで[資格情報]を選択します。 「資格情報の作成」を選択し、アプリケーションのAPIキーを生成します。悪用を防ぐために、おそらくIPアドレスで制限する必要がありますが、今のところ、このデモを試した後は、そのフィールドを空白のままにしてAPIキーを削除してください。
ここにAPIキーをコピーして貼り付けます。

In [ ]:
# chownコマンドを使用して、リポジトリの所有権をユーザーに変更します
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
APIKEY="CHANGE-THIS-KEY"  # APIキーに置き換えます

<b>注：必ずAPIキーを生成し、上記の値を置き換えてください。サンプルキーは機能しません。</b>


<h2>TranslateAPIを呼び出す</h2>

In [3]:
# TranslateAPIの実行
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

# サービスを利用する
inputs = ['is it really this easy?', 'amazing technology', 'wow']
outputs = service.translations().list(source='en', target='ja', q=inputs).execute()

# Print出力
for input, output in zip(inputs, outputs['translations']):
  print("{0} -> {1}".format(input, output['translatedText']))

is it really this easy? -> est-ce vraiment si facile?
amazing technology -> technologie incroyable
wow -> sensationnel


<h2>VisionAPIを呼び出す </h2>

Vision APIは、Cloud Storage内の画像を処理することも、POSTメッセージに直接埋め込むこともできます。 Cloud Storageを使用し、次の画像でOCRを実行します
：<img src = "https://storage.googleapis.com/cloud-training-demos/vision/sign2.jpg" width = "200" />。その写真はhttp://www.publicdomainpictures.net/view-image.php?image=15842からのものです

In [5]:
#VisionAPIの実行
import base64
IMAGE="gs://cloud-training-demos/vision/sign2.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 3,
                }]
            }],
        })
responses = request.execute(num_retries=3)

# `responses`を出力してみましょう
print(responses)

{'responses': [{'textAnnotations': [{'description': '请您爱护和保\n护卫生创建优\n美水环境\n', 'locale': 'zh', 'boundingPoly': {'vertices': [{'y': 104, 'x': 152}, {'y': 104, 'x': 1082}, {'y': 658, 'x': 1082}, {'y': 658, 'x': 152}]}}, {'description': '请', 'boundingPoly': {'vertices': [{'y': 104, 'x': 179}, {'y': 104, 'x': 321}, {'y': 243, 'x': 321}, {'y': 243, 'x': 179}]}}, {'description': '您', 'boundingPoly': {'vertices': [{'y': 108, 'x': 329}, {'y': 108, 'x': 471}, {'y': 243, 'x': 471}, {'y': 243, 'x': 329}]}}, {'description': '爱护', 'boundingPoly': {'vertices': [{'y': 108, 'x': 479}, {'y': 108, 'x': 775}, {'y': 243, 'x': 775}, {'y': 243, 'x': 479}]}}, {'description': '和', 'boundingPoly': {'vertices': [{'y': 111, 'x': 786}, {'y': 111, 'x': 925}, {'y': 243, 'x': 925}, {'y': 243, 'x': 786}]}}, {'description': '保', 'boundingPoly': {'vertices': [{'y': 104, 'x': 933}, {'y': 104, 'x': 1079}, {'y': 239, 'x': 1079}, {'y': 239, 'x': 933}]}}, {'description': '护', 'boundingPoly': {'vertices': [{'y': 300, 'x': 165

In [6]:
foreigntext = responses['responses'][0]['textAnnotations'][0]['description']
foreignlang = responses['responses'][0]['textAnnotations'][0]['locale']

# Let's output the `foreignlang` and `foreigntext`
print(foreignlang, foreigntext)

zh 请您爱护和保
护卫生创建优
美水环境



<h2> 標識を翻訳する </h2>

In [7]:
inputs=[foreigntext]
outputs = service.translations().list(source=foreignlang, target='ja', q=inputs).execute()

# 出力する
for input, output in zip(inputs, outputs['translations']):
  print("{0} -> {1}".format(input, output['translatedText']))

请您爱护和保
护卫生创建优
美水环境
 -> Please care for and protect the health to create a beautiful water environment


<h2>言語APIを使用した感情分析</h2>

Google Cloud Natural Language APIを使用して、いくつかの有名な引用の感情を評価してみましょう。

In [8]:
#Google Cloud Natural LanguageAPIを使用して感情を評価する
lservice = build('language', 'v1beta1', developerKey=APIKEY)
quotes = [
  '成功するためには、途方もない忍耐力、途方もない意志が必要です。',
  'It’s not that I’m so smart, it’s just that I stay with problems longer.',
  'Love is quivering happiness.',
  'Love is of all passions the strongest, for it attacks simultaneously the head, the heart, and the senses.',
  'What difference does it make to the dead, the orphans and the homeless, whether the mad destruction is wrought under the name of totalitarianism or in the holy name of liberty or democracy?',
  'When someone you love dies, and you’re not expecting it, you don’t lose her all at once; you lose her in pieces over a long time — the way the mail stops coming, and her scent fades from the pillows and even from the clothes in her closet and drawers. '
]
for quote in quotes:
# The `documents.analyzeSentiment` method analyzes the sentiment of the provided text.
  response = lservice.documents().analyzeSentiment(
    body={
      'document': {
         'type': 'PLAIN_TEXT',
         'content': quote
      }
    }).execute()
  polarity = response['documentSentiment']['polarity']
  magnitude = response['documentSentiment']['magnitude']

# Lets output the value of each `polarity`, `magnitude` and `quote`
  print('POLARITY=%s MAGNITUDE=%s for %s' % (polarity, magnitude, quote))

POLARITY=1 MAGNITUDE=0.9 for To succeed, you must have tremendous perseverance, tremendous will.
POLARITY=-1 MAGNITUDE=0.5 for It’s not that I’m so smart, it’s just that I stay with problems longer.
POLARITY=1 MAGNITUDE=0.9 for Love is quivering happiness.
POLARITY=1 MAGNITUDE=0.9 for Love is of all passions the strongest, for it attacks simultaneously the head, the heart, and the senses.
POLARITY=1 MAGNITUDE=0.2 for What difference does it make to the dead, the orphans and the homeless, whether the mad destruction is wrought under the name of totalitarianism or in the holy name of liberty or democracy?
POLARITY=-1 MAGNITUDE=0.4 for When someone you love dies, and you’re not expecting it, you don’t lose her all at once; you lose her in pieces over a long time — the way the mail stops coming, and her scent fades from the pillows and even from the clothes in her closet and drawers. 


<h2> Speech API </h2>

Speech APIは、ストリーミングデータ、エンコードされてPOSTメッセージに直接埋め込まれたオーディオコンテンツ、またはCloudStorage上のファイルで機能します。ここでは、この<ahref = "https://storage.googleapis.com/cloud-training-demos/vision/audio.raw">オーディオファイル</a>をクラウドストレージに渡します。

In [10]:
# オーディオファイルを引数として渡すことによるSpeechAPIの使用
sservice = build('speech', 'v1', developerKey=APIKEY)
# `speech.recognize`メソッドは同期音声認識を実行します。
# すべてのオーディオが送信および処理された後、結果を受け取ります。
response = sservice.speech().recognize(
    body={
        'config': {
            'languageCode' : 'en-US',
            'encoding': 'LINEAR16',
            'sampleRateHertz': 16000
        },
        'audio': {
            'uri': 'gs://cloud-training-demos/vision/audio.raw'
            }
        }).execute()

# Let's output the value of `response`
print(response)

{'results': [{'alternatives': [{'confidence': 0.98360395, 'transcript': 'how old is the Brooklyn Bridge'}]}]}


In [11]:
print(response['results'][0]['alternatives'][0]['transcript'])

# 'Confidence`の値を出力してみましょう
print('Confidence=%f' % response['results'][0]['alternatives'][0]['confidence'])

how old is the Brooklyn Bridge
Confidence=0.983604


<h2>クリーンアップ</h2>

<a href="http://console.cloud.google.com/apis"> APIコンソール</a>にアクセスしてAPIキーを削除することを忘れないでください。

必要に応じて、すべてのノートブックをgitにコミットします。

Compute Engine VMでDatalabを実行している場合、またはVMに委任している場合は、課金されないように、Datalabを停止またはシャットダウンすることを忘れないでください。

##チャレンジエクササイズ

これは、ニューヨークのメトロポリタン美術館からのいくつかの肖像画です（これらは、[BigQueryパブリックデータセット]（https://bigquery.cloud.google.com/dataset/bigquery-public-data:the_met）の一部です）：

* gs：//cloud-training-demos/images/met/APS6880.jpg
* gs：//cloud-training-demos/images/met/DP205018.jpg
* gs：//cloud-training-demos/images/met/DP290402.jpg
* gs：//cloud-training-demos/images/met/DP700302.jpg

Vision APIを使用して、これらの画像のどれが幸せな人々を描いているか、どの画像が不幸な人々を描いているかを識別します。

ヒント（ハイライト表示）：<pstyle = "color：white">応答からjoyLikelihoodおよび/またはsorrowLikelihoodを探す必要があります。</ p>

Copyright 2020 Google Inc.
Apacheライセンスバージョン2.0（「ライセンス」）に基づいてライセンス供与されます。ライセンスに準拠する場合を除き、このファイルを使用することはできません。ライセンスのコピーは、次のURLで入手できます。
http://www.apache.org/licenses/LICENSE-2.0
適用法で義務付けられているか、書面で合意されていない限り、ライセンスに基づいて配布されるソフトウェアは、明示または黙示を問わず、いかなる種類の保証または条件もなしに、「現状有姿」で配布されます。ライセンスに基づく許可と制限を規定する特定の言語については、ライセンスを参照してください。